In [6]:
from keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                 include_top=False,
                 )
conv_base.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0         
__________

In [7]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

This notebook relies on you having already split the data in the previous notebook.  
This function is getting the output of the VGG16 convolutional base

In [8]:
base_dir = 'C:\\Users\\zachl\\Documents\\books\\manning\\keras-in-action\\chapter 5\\data\\min'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 4, 4, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode = 'binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        print(i*batch_size, sample_count)
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size: (i+1)* batch_size] = features_batch
        labels[i * batch_size: (i+1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            break
    return features, labels


### Computationally Expensive

In [ ]:
train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

Found 2000 images belonging to 2 classes.
0 2000
20 2000
40 2000
60 2000
80 2000
100 2000
120 2000
140 2000
160 2000
180 2000
200 2000
220 2000
240 2000
260 2000
280 2000
300 2000
320 2000
340 2000
360 2000
380 2000
400 2000
420 2000
440 2000


In [ ]:
train_features = np.reshape(train_features, (2000, 4 * 4 * 512))
validation_features = np.reshape(validation_features, (1000, 4 * 4 * 512))
test_features = np.reshape(test_features, (1000, 4 * 4 * 512))

In [ ]:
from keras import models
from keras import layers
from keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim = 4 * 4 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['acc'])

In [ ]:
history = model.fit(train_features, train_labels,
                   epochs=30,
                   batch_size=20,
                   validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt
epochs = range(0, len(history.history['acc']))
plt.plot(epochs, history.history['acc'], 'bo', label="Training Accuracy")
plt.plot(epochs, history.history['val_acc'], 'b', label="Validation Accuracy")
plt.title("Training and validation accuracy")
plt.figure()
plt.plot(epochs, history.history['loss'], 'bo', label="Training Loss")
plt.plot(epochs, history.history['val_loss'], 'b', label="Validation Loss")
plt.title("Training and validation loss")
plt.show()

In [ ]:
model.save('pretrained_full_train.h5')

The convolutional base output's last axis is the filter dimension.  
The loss is the mean of all values of the 0th filter. Gradient Descent will maximize a blank input image to activate this filter

In [ ]:
def deprocess_image(x):
    x -= x.mean()
    x /= x.std() + 1e-5
    x *= 0.1
    
    x += 0.5
    x = np.clip(x, 0, 1)
    
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
from keras import backend as K
def generate_pattern(l_name, f_index, size=150):
    print(l_name, f_index)
    layer_output = conv_base.get_layer(l_name).output
    loss = K.mean(layer_output[:,:,:,f_index])
    grads = K.gradients(loss, conv_base.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([conv_base.input], [loss, grads])
    input_image_data = np.zeros((1, size, size, 3)) * 20 + 128.
    step = 1.
    for i in range(40):
        loss_value, grads_value = iterate([input_image_data])
        input_image_data += grads_value * step
    return deprocess_image(input_image_data[0])

In [ ]:
import matplotlib.pyplot as plt
filterActivation = generate_pattern('block4_conv1', 5)
print(filterActivation.shape)
plt.matshow(filterActivation[:,:,0])

In [ ]:
layer_name = 'block5_conv1'
amount = 512
layer = conv_base.get_layer(layer_name)
layer_size = layer.output.shape[3]

In [ ]:
import matplotlib.pyplot as plt
subFrom = layer_size - amount
patterns = [generate_pattern(layer_name, i) for i in range(layer_size - subFrom)]

In [ ]:
for pattern in patterns:
    plt.imshow(pattern)
    plt.figure()
    print(pattern.shape)